<a href="https://colab.research.google.com/github/nkcong206/RAG_tutorial/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/e3e66c50-514f-11ef-a9c5-539ef4fa11ba-RAGGraph___Neo4J___CamelAI__2_.png)

Tác giả: [ProtonX Team](https://protonx.io/)

In [7]:
!pip install --upgrade --quiet  langchain sentence_transformers

In [1]:
!pip install -q langchain-chroma
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchainhub
!pip install -q gpt4all

In [6]:
!pip install langchain-huggingface

In [31]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/tuyensinh.txt",encoding="utf8")

Tải dữ liệu từ trang web bất kỳ. Đây là bài viết về Hồ Gươm

In [32]:
document = loader.load()

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = text_splitter.split_documents(document)

In [5]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

In [34]:
from langchain_huggingface import HuggingFaceEmbeddings

# model = SentenceTransformer("dangvantuan/vietnamese-embedding")

model_name="bkai-foundation-models/vietnamese-bi-encoder"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [45]:
# model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
# gpt4all_kwargs = {'allow_download': 'True'}
# embeddings = GPT4AllEmbeddings(
#     model_name=model_name,
#     gpt4all_kwargs=gpt4all_kwargs
# )

In [ ]:
db = Chroma(
    collection_name="new_collection",
    embedding_function=embeddings,
    persist_directory="/content/chroma_db"
)
db.add_documents(documents=docs)

In [21]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [22]:
!curl https://ollama.ai/install.sh | sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  45449      0 --:--:-- --:--:-- --:--:-- 45568
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Chạy server OLLAMA ở một subprocess khác.

In [23]:
import subprocess
import time
import threading

# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")


Ollama server is running in the background


In [15]:
# Chạy dòng code này trên terminal nếu cell trên không hoạt động
#!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: address already in use


Tải llama 3.1

In [24]:
!ollama pull ontocord/vistral

[GIN] 2024/08/13 - 17:07:31 | 200 |     141.456µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling d7a2da04a18e... 100% ▕▏ 4.1 GB                         
pulling 5351f5046769... 100% ▕▏  112 B                         
pulling fa304d675061... 100% ▕▏   91 B                         
pulling 8aee4aaa236b... 100% ▕▏  409 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [25]:
llm = ChatOllama(model="ontocord/vistral")
import os
os.environ["LANGCHAIN_API_KEY"] = "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"
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="/content/chroma_db", embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [26]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=db.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

Đặt câu hỏi truy vấn

In [29]:
question = "Thời gian tiếp nhận đăng ký học bổng?"
result = qa_chain({"query": question ,})

[GIN] 2024/08/13 - 17:08:38 | 200 |  5.742294029s |       127.0.0.1 | POST     "/api/chat"


In [30]:
result

{'query': 'Thời gian tiếp nhận đăng ký học bổng?',
 'result': '\nThời hạn nộp đơn đăng ký học bổng là trước ngày 15 tháng 8 năm 2024, với kết quả dự kiến sẽ được thông báo trong tháng 9/2024. Học bổng chỉ xét cho sinh viên đã hoàn thành thủ tục đăng ký đúng thời gian và đầy đủ. Nếu bạn có bất kỳ câu hỏi nào khác, vui lòng liên hệ Phòng Công tác Sinh viên tại Phòng 606, Nhà A21, 18 Hoàng Quốc Việt, Cầu Giấy, Hà Nội hoặc email scholarship@usth.edu.vn. Bạn cũng có thể gọi đến số điện thoại +84-2437917201 / 0593002509 để được hỗ trợ thêm thông tin về học bổng này. '}